In [ ]:
from animal_shelter import AnimalShelter
import dash
import dash_leaflet as dl
import plotly.express as px
from dash import dash_table, html, dcc
from dash.dependencies import Input, Output, State
import pandas as pd
import base64

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "admin"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object and accept projection json input
#df = pd.DataFrame.from_records(shelter.readAll({}))
df = pd.DataFrame.from_records(shelter.readAll({}))

#########################
# Dashboard Layout / View
#########################
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'file.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
    html.Img(src='data:image/png;base64, {}'.format(encoded_image.decode()),
        style={
            'height' : '25%',
            'width'  : '25%',
            'float'  : 'left',
            'position': 'relative',
            'padding-top': 0,
            'padding-left':0,
        })
    ], href='https://www.snhu.edu'),
html.Center(html.B(html.H1('Search and Rescue Candidate Locator'))),
html.Br(),
html.Header("Tenny Vongtip"),
html.Hr(),
dcc.Dropdown(
id='select-categories',
options=[
    {'label': 'All', 'value': 'All'},
    {'label': 'Water Rescue', 'value': 'WTR'},
    {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
    {'label': 'Disaster/Individual Tracking', 'value': 'DIT'}
],
    value='All'

),

html.Div([
    html.Label(['Pie chart']),
    dcc.Dropdown(
        id='my_dropdown',
        options=[
            {'label': 'Animal Breed', 'value' : 'breed' },
            {'label': 'Animal Type', 'value' : 'animal_type' },
            {'label': 'Animal Color', 'value' : 'color' }
],
        value='animal_type',
        multi=False,
        clearable=False,
        style={"width": "50%"}
    ),
    ]),
    html.Div([
        dcc.Graph(id='the_graph')
    ]),
    ])

dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),  # imports DATA
        editable=False,  #start of data Table
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size =10,
    ),
html.Br(),
html.Hr(),
html.Div(className='row',
        style={'display' : 'flex'},
        children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
           id='map-id',
           className='col s12 m6',
            ),

        ])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output(component_id='the_graph', component_property='figure'),
    [Input(component_id='my_dropdown', component_property='value')]
)
def update_graph(my_dropdown):
    dff = df
    piechart=px.pie(
        data_frame=dff,
        names=my_dropdown,
        hole=.3,
    )
    return piechart

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#This callback will highlight a row on the data table when the user selects it
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('select-categories', 'value')])
def update_dashboard(value):
        ### FIX ME Add code to filter interactive data table with MongoDB queries
        if value == 'All':
            df = pd.DataFrame.from_records(shelter.readAll({}))
        elif value == 'WTR':
            df = pd.DataFrame(list(shelter.readAll({"breed" : {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
        elif value == 'MWR':
            df = pd.DataFrame(list(shelter.readAll({"breed" : {"$in": ["German Shephard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
        elif value == 'DIT':
            df = pd.DataFrame(list(shelter.readAll({"breed" : {"$in": ["Doberman Pinscher", "German Shephard", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}})))

        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')


        return data,columns


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]

if __name__ =='__main__':
    app.run(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [10/Dec/2022 20:03:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:03:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:03:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:03:18] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:03:18] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2022 20:03:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2022 20:03:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2022 20:03:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:03:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:03:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 20:13:39] "POST /_dash-update-component HTTP/1.1" 200